In [ ]:
from pathlib import Path
import shutil

BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250721/20250721_good_data")
DEST = BASE / "class3"
DEST.mkdir(exist_ok=True)

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".JPG", ".JPEG", ".PNG", ".BMP"}

def has_class3(txt_path: Path) -> bool:
    with open(txt_path, "r") as f:
        for line in f:
            if line.strip() and line.split()[0] == "3":
                return True
    return False

def find_image(label_path: Path):
    stem = label_path.stem
    parent = label_path.parent
    for ext in IMG_EXTS:
        cand = parent / f"{stem}{ext}"
        if cand.exists():
            return cand
    return None

moved_txt = 0
moved_img = 0

# 핵심: 처음에 '고정된 리스트'를 만들고, DEST 아래는 제외
txt_list = [p for p in BASE.rglob("*.txt") if DEST not in p.parents]

for txt in txt_list:
    if has_class3(txt):
        img = find_image(txt)

        # 라벨 이동
        if txt.parent != DEST:
            shutil.move(str(txt), str(DEST / txt.name))
            moved_txt += 1

        # 이미지도 있으면 이동
        if img and img.parent != DEST:
            shutil.move(str(img), str(DEST / img.name))
            moved_img += 1

print(f"[완료] class 3 라벨 {moved_txt}개, 이미지 {moved_img}개를 {DEST}로 이동했습니다.")

In [3]:
# ==========================
# Delete all .txt label files except classes.txt
# ==========================
from pathlib import Path
import os

folders = [
    "/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_bad_data",
    "/home/dw/ws_job_msislab/Golf_Project/data/20250721/20250721_bad_data",
    "/home/dw/ws_job_msislab/Golf_Project/data/20250725/20250725_bad_data",
    "/home/dw/ws_job_msislab/Golf_Project/data/20250929/20250929_bad_data",
    "/home/dw/ws_job_msislab/Golf_Project/data/20250930/20250930_bad_data",
    "/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_bad_data2",
]

deleted = 0
for f in folders:
    path = Path(f)
    if not path.exists():
        print(f"[WARN] 경로 없음: {f}")
        continue

    for txt in path.rglob("*.txt"):
        # === classes.txt는 삭제하지 않음 ===
        if txt.name == "classes.txt":
            continue
        try:
            os.remove(txt)
            deleted += 1
        except Exception as e:
            print(f"[ERROR] 삭제 실패: {txt} ({e})")

print(f"[RESULT] 총 {deleted}개의 txt 파일 삭제 완료 (classes.txt 제외)")

[RESULT] 총 0개의 txt 파일 삭제 완료 (classes.txt 제외)


### weights

In [ ]:
# === YOLO 자동라벨링: 0/1 혼동이면 3으로 기록(널널 버전) + 최종 요약만 출력 ===
from ultralytics import YOLO
from pathlib import Path
from typing import List, Tuple
from collections import defaultdict

# ================== 경로/설정 ==================
MODEL_PT = "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20250920_merge_data_yolov8s_sgd/weights/best.pt"
IMG_DIR  = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250721/20250721_bad_data")

# 클래스 매핑 (순서 고정)
# 0: Divot, 1: Fixed_Divot, 2: Diseased_Grass, 3: Confused_Object,
# 4: Pole, 5: Sprinkler, 6: Drain, 7: Golf ball
MIN_CONF_PER_CLASS = {
    0: 0.45,
    1: 0.45,
    2: 0.60,
    4: 0.80,
    5: 0.60,
    6: 0.60,
    7: 0.60,
}

# 혼동 규칙 파라미터(더 느슨하게)
IOU_MATCH_THRESH = 0.40    # 같은 물체로 간주할 IoU 기준
DELTA_CONF      = 0.40     # |conf0 - conf1| < 0.40 이면 3으로
BASE_CONF       = 0.01     # 넓게 잡고 후처리에서 필터
IMG_EXTS        = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

# === 혼동 시 최소점수 요구 여부(널널 모드: False) ===
REQUIRE_MINCONF_FOR_CONFUSION = False    # 혼동 보낼 때 min conf 필요 여부
REQUIRE_BOTH_MINCONF          = False    # True면 c0,c1 둘 다 넘어야 혼동 허용(보수적)

# ==============================================

def iou_xyxy(a, b):
    x1 = max(a[0], b[0]); y1 = max(a[1], b[1])
    x2 = min(a[2], b[2]); y2 = min(a[3], b[3])
    iw = max(0.0, x2 - x1); ih = max(0.0, y2 - y1)
    inter = iw * ih
    if inter <= 0: return 0.0
    area_a = (a[2]-a[0])*(a[3]-a[1]); area_b = (b[2]-b[0])*(b[3]-b[1])
    return inter / (area_a + area_b - inter + 1e-9)

def collect_images(root: Path) -> List[Path]:
    return [p for p in root.rglob("*") if p.suffix.lower() in IMG_EXTS]

def ensure_label_path(img_path: Path) -> Path:
    parts = list(img_path.parts)
    if "images" in parts:
        i = parts.index("images")
        parts[i] = "labels"
        label_dir = Path(*parts[:-1])
    else:
        label_dir = img_path.parent / "labels"
    label_dir.mkdir(parents=True, exist_ok=True)
    return label_dir / (img_path.stem + ".txt")

def write_yolo_txt(label_path: Path, rows: List[Tuple[int, float, float, float, float]]):
    if not rows:
        if label_path.exists(): label_path.unlink()
        return False  # 아무 라벨도 쓰지 않음
    with open(label_path, "w") as f:
        for cls, cx, cy, w, h in rows:
            f.write(f"{cls} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")
    return True  # 라벨 기록됨

def detect(model: YOLO, img: Path, classes=None):
    r = model.predict(source=str(img), classes=classes, conf=BASE_CONF, verbose=False)[0]
    if r.boxes is None or len(r.boxes) == 0: return []
    xyxy  = r.boxes.xyxy.cpu().numpy()
    confs = r.boxes.conf.cpu().numpy()
    clses = r.boxes.cls.cpu().numpy().astype(int)
    xywhn = r.boxes.xywhn.cpu().numpy()
    return [
        {"cls": int(clses[i]), "conf": float(confs[i]),
         "xyxy": tuple(map(float, xyxy[i])), "xywhn": tuple(map(float, xywhn[i]))}
        for i in range(len(clses))
    ]

def main():
    model = YOLO(MODEL_PT)
    images = collect_images(IMG_DIR)

    # --- 통계 집계 변수 ---
    per_class_counts = defaultdict(int)   # 클래스별 라벨 수
    images_labeled   = 0                  # 라벨이 1개 이상 기록된 이미지 수
    total_labels     = 0                  # 총 라벨 수

    for img in images:
        # 기타(2,4,5,6,7)
        other_boxes = detect(model, img, classes=[2,4,5,6,7])

        # 0-only, 1-only
        divot_boxes = detect(model, img, classes=[0])
        fixed_boxes = detect(model, img, classes=[1])

        used0, used1 = set(), set()
        keep_01, confused_rows = [], []

        for i, b0 in enumerate(divot_boxes):
            best_j, best_iou = -1, 0.0
            for j, b1 in enumerate(fixed_boxes):
                if j in used1: continue
                iou = iou_xyxy(b0["xyxy"], b1["xyxy"])
                if iou > best_iou:
                    best_iou, best_j = iou, j

            if best_j >= 0 and best_iou >= IOU_MATCH_THRESH:
                c0, c1 = b0["conf"], fixed_boxes[best_j]["conf"]

                # ---- 혼동 허용 여부(최소점수 조건 스위치) ----
                allow_by_minconf = True
                if REQUIRE_MINCONF_FOR_CONFUSION:
                    if REQUIRE_BOTH_MINCONF:
                        allow_by_minconf = (c0 >= MIN_CONF_PER_CLASS[0] and c1 >= MIN_CONF_PER_CLASS[1])
                    else:
                        allow_by_minconf = (c0 >= MIN_CONF_PER_CLASS[0] or  c1 >= MIN_CONF_PER_CLASS[1])

                # === 핵심 규칙: 점수 차 < DELTA_CONF → 3번으로 라벨링 ===
                if abs(c0 - c1) < DELTA_CONF and allow_by_minconf:
                    winner = b0 if c0 >= c1 else fixed_boxes[best_j]
                    cx, cy, w, h = winner["xywhn"]
                    confused_rows.append((3, cx, cy, w, h))
                    used0.add(i); used1.add(best_j)
                else:
                    # 혼동 아니면 승자만 채택(승자 점수가 min conf 넘을 때)
                    winner = b0 if c0 >= c1 else fixed_boxes[best_j]
                    if winner["conf"] >= MIN_CONF_PER_CLASS[winner["cls"]]:
                        keep_01.append(winner)
                    used0.add(i); used1.add(best_j)

        # 매칭 실패 0/1도 임계 넘으면 유지
        for i, b0 in enumerate(divot_boxes):
            if i in used0: continue
            if b0["conf"] >= MIN_CONF_PER_CLASS[0]:
                keep_01.append(b0)
        for j, b1 in enumerate(fixed_boxes):
            if j in used1: continue
            if b1["conf"] >= MIN_CONF_PER_CLASS[1]:
                keep_01.append(b1)

        # 최종 저장 행 구성
        rows: List[Tuple[int, float, float, float, float]] = []
        for b in keep_01:
            cx, cy, w, h = b["xywhn"]
            rows.append((b["cls"], cx, cy, w, h))
        rows.extend(confused_rows)
        for b in other_boxes:
            thr = MIN_CONF_PER_CLASS.get(b["cls"], 0.0)
            if b["conf"] >= thr:
                cx, cy, w, h = b["xywhn"]
                rows.append((b["cls"], cx, cy, w, h))

        label_path = ensure_label_path(img)
        wrote = write_yolo_txt(label_path, rows)

        # --- 통계 집계 ---
        if wrote:
            images_labeled += 1
            total_labels   += len(rows)
            for cls, *_ in rows:
                per_class_counts[cls] += 1

    # === 최종 요약 출력 ===
    print("========== Auto-Labeling Summary ==========")
    print(f"Images scanned     : {len(images)}")
    print(f"Images labeled     : {images_labeled}")
    print(f"Total labels saved : {total_labels}")
    print("-------------------------------------------")
    # 클래스 ID 순서대로 깔끔히 출력
    class_names = {
        0: "Divot",
        1: "Fixed_Divot",
        2: "Diseased_Grass",
        3: "Confused_Object",
        4: "Pole",
        5: "Sprinkler",
        6: "Drain",
        7: "Golf ball",
    }
    for cid in range(0, 8):
        cnt = per_class_counts.get(cid, 0)
        print(f"Class {cid:>1} ({class_names[cid]:>16}): {cnt}")
    print("===========================================")

if __name__ == "__main__":
    main()

### engine버전

In [ ]:
# === YOLO 자동라벨링 (ENGINE): 진행상황은 N장마다 출력, 최종 요약만 자세히 ===
from ultralytics import YOLO
from pathlib import Path
from typing import List, Tuple
from collections import defaultdict
import sys

# ================== 경로/설정 ==================
ENGINE    = "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251107_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251107/weights/best_fp16_bs1_640px_static.engine"
ENGINE_IMGSZ = 640
ENGINE_BATCH = 1

IMG_DIR  = Path("/home/dw/ws_job_msislab/Golf_Project/data/leaves_background")

# 진행 출력 주기(장 수)
PROGRESS_EVERY = 50   # ✅ 필요하면 20/100 등으로 변경

# 클래스 매핑 (순서 고정)
MIN_CONF_PER_CLASS = {0:0.40, 1:0.40, 2:0.50, 4:0.50, 5:0.50, 6:0.50, 7:0.50}
#MIN_CONF_PER_CLASS = {0:0.001, 1:0.001, 2:0.001, 4:0.001, 5:0.001, 6:0.001, 7:0.001}
# 혼동 규칙(널널)
IOU_MATCH_THRESH = 0.5
DELTA_CONF      = 0.01
BASE_CONF       = 0.01
IMG_EXTS        = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

REQUIRE_MINCONF_FOR_CONFUSION = False
REQUIRE_BOTH_MINCONF          = False

def iou_xyxy(a, b):
    x1 = max(a[0], b[0]); y1 = max(a[1], b[1])
    x2 = min(a[2], b[2]); y2 = min(a[3], b[3])
    iw = max(0.0, x2 - x1); ih = max(0.0, y2 - y1)
    inter = iw * ih
    if inter <= 0: return 0.0
    area_a = (a[2]-a[0])*(a[3]-a[1]); area_b = (b[2]-b[0])*(b[3]-b[1])
    return inter / (area_a + area_b - inter + 1e-9)

def collect_images(root: Path):
    return [p for p in root.rglob("*") if p.suffix.lower() in IMG_EXTS]

def ensure_label_path(img_path: Path) -> Path:
    parts = list(img_path.parts)
    if "images" in parts:
        i = parts.index("images"); parts[i] = "labels"
        label_dir = Path(*parts[:-1])
    else:
        label_dir = img_path.parent / "labels"
    label_dir.mkdir(parents=True, exist_ok=True)
    return label_dir / (img_path.stem + ".txt")

def write_yolo_txt(label_path: Path, rows: List[Tuple[int, float, float, float, float]]):
    if not rows:
        if label_path.exists(): label_path.unlink()
        return False
    with open(label_path, "w") as f:
        for cls, cx, cy, w, h in rows:
            f.write(f"{cls} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")
    return True

def _xyxy_to_xywhn(xyxy, img_w, img_h):
    x1, y1, x2, y2 = xyxy
    w = max(0.0, x2 - x1); h = max(0.0, y2 - y1)
    cx = x1 + w / 2.0; cy = y1 + h / 2.0
    return (cx/(img_w+1e-9), cy/(img_h+1e-9), w/(img_w+1e-9), h/(img_h+1e-9))

def detect(model: YOLO, img: Path, classes=None):
    r = model.predict(
        source=str(img), classes=classes, conf=BASE_CONF,
        imgsz=ENGINE_IMGSZ, device=0, verbose=False
    )[0]
    if r.boxes is None or len(r.boxes) == 0:
        return []
    xyxy  = r.boxes.xyxy.cpu().numpy()
    confs = r.boxes.conf.cpu().numpy()
    clses = r.boxes.cls.cpu().numpy().astype(int)
    try:
        xywhn = r.boxes.xywhn.cpu().numpy(); has_xywhn = True
    except Exception:
        has_xywhn = False
    H, W = r.orig_shape
    out = []
    for i in range(len(clses)):
        if has_xywhn:
            cx, cy, w, h = map(float, xywhn[i])
        else:
            cx, cy, w, h = _xyxy_to_xywhn(tuple(map(float, xyxy[i])), W, H)
        out.append({"cls": int(clses[i]), "conf": float(confs[i]),
                    "xyxy": tuple(map(float, xyxy[i])),
                    "xywhn": (cx, cy, w, h)})
    return out

def main():
    model = YOLO(ENGINE)
    images = collect_images(IMG_DIR)

    per_class_counts = defaultdict(int)
    images_labeled   = 0
    total_labels     = 0

    total = len(images)
    # 최소 안내 한 줄만 — 총 장수 알려주기
    print(f"[Start] Total images: {total}  (progress every {PROGRESS_EVERY})"); sys.stdout.flush()

    for idx, img in enumerate(images, start=1):
        try:
            other_boxes = detect(model, img, classes=[2,4,5,6,7])
            divot_boxes = detect(model, img, classes=[0])
            fixed_boxes = detect(model, img, classes=[1])

            used0, used1 = set(), set()
            keep_01, confused_rows = [], []

            for i, b0 in enumerate(divot_boxes):
                best_j, best_iou = -1, 0.0
                for j, b1 in enumerate(fixed_boxes):
                    if j in used1: continue
                    iou = iou_xyxy(b0["xyxy"], b1["xyxy"])
                    if iou > best_iou:
                        best_iou, best_j = iou, j

                if best_j >= 0 and best_iou >= IOU_MATCH_THRESH:
                    c0, c1 = b0["conf"], fixed_boxes[best_j]["conf"]
                    allow_by_minconf = True
                    if REQUIRE_MINCONF_FOR_CONFUSION:
                        if REQUIRE_BOTH_MINCONF:
                            allow_by_minconf = (c0 >= MIN_CONF_PER_CLASS[0] and c1 >= MIN_CONF_PER_CLASS[1])
                        else:
                            allow_by_minconf = (c0 >= MIN_CONF_PER_CLASS[0] or  c1 >= MIN_CONF_PER_CLASS[1])

                    if abs(c0 - c1) < DELTA_CONF and allow_by_minconf:
                        winner = b0 if c0 >= c1 else fixed_boxes[best_j]
                        cx, cy, w, h = winner["xywhn"]
                        confused_rows.append((3, cx, cy, w, h))
                        used0.add(i); used1.add(best_j)
                    else:
                        winner = b0 if c0 >= c1 else fixed_boxes[best_j]
                        if winner["conf"] >= MIN_CONF_PER_CLASS.get(winner["cls"], 0.0):
                            keep_01.append(winner)
                        used0.add(i); used1.add(best_j)

            for i, b0 in enumerate(divot_boxes):
                if i in used0: continue
                if b0["conf"] >= MIN_CONF_PER_CLASS[0]:
                    keep_01.append(b0)
            for j, b1 in enumerate(fixed_boxes):
                if j in used1: continue
                if b1["conf"] >= MIN_CONF_PER_CLASS[1]:
                    keep_01.append(b1)

            rows: List[Tuple[int, float, float, float, float]] = []
            for b in keep_01:
                cx, cy, w, h = b["xywhn"]; rows.append((b["cls"], cx, cy, w, h))
            rows.extend(confused_rows)
            for b in other_boxes:
                thr = MIN_CONF_PER_CLASS.get(b["cls"], 0.0)
                if b["conf"] >= thr:
                    cx, cy, w, h = b["xywhn"]; rows.append((b["cls"], cx, cy, w, h))

            label_path = ensure_label_path(img)
            wrote = write_yolo_txt(label_path, rows)

            if wrote:
                images_labeled += 1
                total_labels   += len(rows)
                for cls, *_ in rows:
                    per_class_counts[cls] += 1

        except Exception as e:
            # 에러는 한 줄만 남기고 계속 진행
            sys.stdout.write(f"\n[Skip] #{idx}: {img.name} ({e})\n"); sys.stdout.flush()

        # ✅ 진행 상황: N장마다만 출력
        if idx % PROGRESS_EVERY == 0 or idx == total:
            sys.stdout.write(f"[Progress] {idx}/{total}\n"); sys.stdout.flush()

    # === 최종 요약 출력 ===
    print("========== Auto-Labeling Summary (TensorRT ENGINE) ==========")
    print(f"Engine imgsz/batch : {ENGINE_IMGSZ}/{ENGINE_BATCH}")
    print(f"Images scanned     : {total}")
    print(f"Images labeled     : {images_labeled}")
    print(f"Total labels saved : {total_labels}")
    print("-------------------------------------------------------------")
    class_names = {0:"Divot",1:"Fixed_Divot",2:"Diseased_Grass",3:"Confused_Object",
                   4:"Pole",5:"Sprinkler",6:"Drain",7:"Golf ball"}
    for cid in range(0, 8):
        cnt = per_class_counts.get(cid, 0)
        print(f"Class {cid:>1} ({class_names[cid]:>16}): {cnt}")
    print("=============================================================")

if __name__ == "__main__":
    main()



WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
[Start] Total images: 3045  (progress every 50)
Loading /home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251017/20251017_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251020_only_sep/weights/best_fp16_bs1_640px_static.engine for TensorRT inference...
[11/21/2025-11:44:43] [TRT] [I] Loaded engine size: 25 MiB
[11/21/2025-11:44:43] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +17, now: CPU 0, GPU 38 (MiB)
[Progress] 50/3045
[Progress] 100/3045
[Progress] 150/3045
[Progress] 200/3045
[Progress] 250/3045
[Progress] 300/3045
[Progress] 350/3045
[Progress] 400/3045
[Progress] 450/3045
[Progress] 500/3045
[Progress] 550/3045
[Progress] 600/3045
[Progress] 650/3045
[Progress] 700/3045
[Progress] 750/3045
[Progress] 800/3045
[Progress] 850/3045
[Progre

### weight 각각 버전

In [ ]:
# === YOLOv8 자동 라벨링 (클래스별 서로 다른 confidence 적용 + 지정 순서로 저장) ===
from ultralytics import YOLO
from pathlib import Path
import cv2
import math

# ====== 경로 설정 ======
WEIGHT = Path("/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20250912_merge_data_yolov8s_safe/weights/last.pt")
IMG_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250725/20250725_bad_data")
IMG_EXTS = [".jpg", ".jpeg", ".png"]

# ====== 원하는 "저장용 클래스 순서" (요청하신 순서) ======
desired_order = [
    "Divot",
    "Fixed_Divot",
    "Diseased_Grass",  # 사용자가 제공한 철자 그대로 둠
    "Confused_Object",
    "Pole",
    "Sprinkler",
    "Drain",
    "Golf ball",
]

# ====== 클래스별 confidence threshold (원하면 자유롭게 조정) ======
# 키는 "클래스 이름" (위 desired_order 기준), 값은 임계값(0~1)
per_class_conf = {
    "Divot":            0.50,
    "Fixed_Divot":      0.50,
    "Diseased_Grass":    0.75,
    "Confused_Object":  0.50,
    "Pole":             0.50,
    "Sprinkler":        0.50,
    "Drain":            0.50,
    "Golf ball":        0.50,
}
DEFAULT_CONF = 0.5  # 사전에 없는 클래스 이름이 들어올 경우 기본값

# ====== 모델 로드 ======
model = YOLO(str(WEIGHT))
model_names = model.names  # ex) {0: 'Divot', 1: 'Fixed_Divot', ...} 또는 list
if isinstance(model_names, dict):
    idx2name = {int(k): v for k, v in model_names.items()}
else:
    idx2name = {i: n for i, n in enumerate(model_names)}

# ====== "모델 클래스 인덱스" -> "저장용 클래스 인덱스" 매핑 만들기 ======
# 1) 이름이 정확히 매칭되면 그 이름의 위치(desired_order index)로 매핑
# 2) 이름이 매칭 안 되면 경고를 남기고 "동일 인덱스"로 저장(최소한 깨지지 않게)
name2desiredidx = {n: i for i, n in enumerate(desired_order)}
modelidx_to_saveidx = {}
unmatched = []

for midx, mname in idx2name.items():
    if mname in name2desiredidx:
        modelidx_to_saveidx[midx] = name2desiredidx[mname]
    else:
        unmatched.append((midx, mname))
        modelidx_to_saveidx[midx] = midx  # fallback: 인덱스 그대로

if unmatched:
    print("[WARN] 아래 모델 클래스 이름은 desired_order와 매칭되지 않아 '동일 인덱스'로 저장합니다:")
    for midx, mname in unmatched:
        print(f"  - model idx {midx}: '{mname}'")

# ====== 이미지 수집 ======
img_files = [p for p in IMG_DIR.iterdir() if p.suffix.lower() in IMG_EXTS]
print(f"[INFO] Found {len(img_files)} images")

saved_box_count = 0
saved_per_class = {i: 0 for i in range(len(desired_order))}

for img_path in img_files:
    # 예측: 여기서는 conf를 낮게(예: 0.01) 주고, 필터링은 아래에서 클래스별로 수행
    results = model.predict(source=str(img_path), conf=0.01, save=False, verbose=False)

    boxes = results[0].boxes
    if boxes is None or len(boxes) == 0:
        continue

    img = cv2.imread(str(img_path))
    if img is None:
        print(f"[WARN] 이미지 열기 실패: {img_path}")
        continue
    h, w = img.shape[:2]

    # 라벨 파일 열기(필요할 때만 생성)
    lines = []
    for b in boxes:
        cls_idx = int(b.cls[0].item())
        conf = float(b.conf[0].item())

        # 모델 클래스 이름을 가져와 per-class threshold 선택
        cls_name = idx2name.get(cls_idx, None)
        # 저장용 클래스 인덱스(요청 순서 기준)
        save_cls_idx = modelidx_to_saveidx.get(cls_idx, cls_idx)

        # 클래스 이름 기반 임계값(없으면 기본값)
        thr = per_class_conf.get(cls_name, DEFAULT_CONF)

        if conf < thr:
            continue

        # xyxy -> xywh (normalized)
        x1, y1, x2, y2 = b.xyxy[0]
        cx = ((x1 + x2) / 2) / w
        cy = ((y1 + y2) / 2) / h
        bw = (x2 - x1) / w
        bh = (y2 - y1) / h

        # 클램프(숫자 오차 대비)
        def clamp(v): 
            return max(0.0, min(1.0, float(v)))
        cx, cy, bw, bh = map(clamp, (cx, cy, bw, bh))

        # 라벨 라인(요청: "저장용 클래스 순서"의 인덱스로 기록)
        lines.append(f"{save_cls_idx} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}")

        if 0 <= save_cls_idx < len(desired_order):
            saved_per_class[save_cls_idx] += 1
        saved_box_count += 1

    if lines:
        with open(img_path.with_suffix(".txt"), "w") as f:
            f.write("\n".join(lines))

# ====== 리포트 ======
print("====================================")
print(f"[RESULT] 총 이미지 수         : {len(img_files)}")
print(f"[RESULT] 저장된 라벨(박스) 수 : {saved_box_count}")
print("[RESULT] 클래스별 저장된 박스 수 (저장용 순서 기준):")
for i, name in enumerate(desired_order):
    print(f"  - {i:02d} {name:15s}: {saved_per_class.get(i,0)}")
print("====================================")
print("[NOTE] per-class threshold:", per_class_conf)

### 실제 비쥬얼 라이즈 모드

In [11]:
# === YOLO 자동라벨링 + 시각화 (Divot 강조) ===
from ultralytics import YOLO
from pathlib import Path
from typing import List, Tuple
from collections import defaultdict
import cv2, sys

# ================== 경로/설정 ==================
ENGINE = "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251124/20251107_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251122/weights/best_fp16_bs1_640px_static.engine"
ENGINE_IMGSZ = 640
IMG_DIR = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_test/leave_test_2")

PROGRESS_EVERY = 50
BASE_CONF = 0.001
IOU_MATCH_THRESH = 0.5
DELTA_CONF = 0.001

# 클래스별 최소 conf
MIN_CONF_PER_CLASS = {0:0.5, 1:0.5, 2:0.5, 4:0.5, 5:0.5, 6:0.5, 7:0.5}

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}
REQUIRE_MINCONF_FOR_CONFUSION = False
REQUIRE_BOTH_MINCONF = False

# 시각화 저장 폴더
VIZ_DIR = IMG_DIR / "labels_viz"
VIZ_DIR.mkdir(parents=True, exist_ok=True)

# 클래스 이름 / 색상
CLASS_NAMES = {
    0:"Divot",1:"Fixed_Divot",2:"Diseased_Grass",3:"Confused_Object",
    4:"Pole",5:"Sprinkler",6:"Drain",7:"Golf ball"
}

# Divot 강조 컬러 (BGR)
CLASS_COLORS = {
    0: (0, 0, 255),       # 🔴 Divot - 선명한 빨강 (가장 강조)
    1: (0, 140, 255),     # 🟠 Fixed_Divot - 주황
    2: (50, 205, 50),     # 🟢 Diseased_Grass - 연초록
    3: (230, 230, 230),   # ⚪ Confused_Object - 회백
    4: (255, 0, 0),       # 🔵 Pole - 파랑
    5: (180, 0, 255),     # 🟣 Sprinkler - 보라
    6: (42, 42, 165),     # 🟤 Drain - 갈색톤
    7: (0, 255, 255)      # 🟡 Golf ball - 노랑
}

# ================================================

def iou_xyxy(a, b):
    x1 = max(a[0], b[0]); y1 = max(a[1], b[1])
    x2 = min(a[2], b[2]); y2 = min(a[3], b[3])
    iw = max(0.0, x2 - x1); ih = max(0.0, y2 - y1)
    inter = iw * ih
    if inter <= 0: return 0.0
    area_a = (a[2]-a[0])*(a[3]-a[1]); area_b = (b[2]-b[0])*(b[3]-b[1])
    return inter / (area_a + area_b - inter + 1e-9)

def collect_images(root: Path):
    return [p for p in root.rglob("*") if p.suffix.lower() in IMG_EXTS]

def ensure_label_path(img_path: Path) -> Path:
    label_dir = img_path.parent / "labels"
    label_dir.mkdir(parents=True, exist_ok=True)
    return label_dir / (img_path.stem + ".txt")

def write_yolo_txt(label_path: Path, rows: List[Tuple[int, float, float, float, float]]):
    if not rows:
        if label_path.exists(): label_path.unlink()
        return False
    with open(label_path, "w") as f:
        for cls, cx, cy, w, h in rows:
            f.write(f"{cls} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")
    return True

def _xyxy_to_xywhn(xyxy, img_w, img_h):
    x1, y1, x2, y2 = xyxy
    w = max(0.0, x2 - x1); h = max(0.0, y2 - y1)
    cx = x1 + w / 2.0; cy = y1 + h / 2.0
    return (cx/(img_w+1e-9), cy/(img_h+1e-9), w/(img_w+1e-9), h/(img_h+1e-9))

def detect(model: YOLO, img: Path, classes=None):
    r = model.predict(source=str(img), classes=classes, conf=BASE_CONF,
                      imgsz=ENGINE_IMGSZ, device=0, verbose=False)[0]
    if r.boxes is None or len(r.boxes) == 0:
        return []
    xyxy  = r.boxes.xyxy.cpu().numpy()
    confs = r.boxes.conf.cpu().numpy()
    clses = r.boxes.cls.cpu().numpy().astype(int)
    H, W = r.orig_shape
    out = []
    for i in range(len(clses)):
        cx, cy, w, h = _xyxy_to_xywhn(tuple(map(float, xyxy[i])), W, H)
        out.append({"cls": int(clses[i]), "conf": float(confs[i]),
                    "xyxy": tuple(map(float, xyxy[i])),
                    "xywhn": (cx, cy, w, h)})
    return out

def draw_boxes(img_path: Path, boxes: List[dict], out_path: Path):
    img = cv2.imread(str(img_path))
    if img is None:
        return
    for b in boxes:
        cls, conf = b["cls"], b["conf"]
        x1, y1, x2, y2 = map(int, b["xyxy"])
        color = CLASS_COLORS.get(cls, (128,128,128))
        # 🔹 실제 confidence 점수로 표시 (0.00 형식)
        label = f"{CLASS_NAMES.get(cls,'?')} {conf:.4f}"
        thickness = 2  # ✅ 모든 클래스 동일 두께
        cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)
        (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        cv2.rectangle(img, (x1, y1 - th - 6), (x1 + tw + 4, y1), color, -1)
        cv2.putText(img, label, (x1 + 2, y1 - 4),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 2)
    cv2.imwrite(str(out_path), img)

def main():
    model = YOLO(ENGINE)
    images = collect_images(IMG_DIR)
    per_class_counts = defaultdict(int)
    images_labeled, total_labels = 0, 0
    total = len(images)

    print(f"[Start] Total images: {total} (progress every {PROGRESS_EVERY})")

    for idx, img in enumerate(images, start=1):
        try:
            other_boxes = detect(model, img, classes=[2,4,5,6,7])
            divot_boxes = detect(model, img, classes=[0])
            fixed_boxes = detect(model, img, classes=[1])

            used0, used1 = set(), set()
            keep_01, confused_rows = [], []

            for i, b0 in enumerate(divot_boxes):
                best_j, best_iou = -1, 0.0
                for j, b1 in enumerate(fixed_boxes):
                    if j in used1: continue
                    iou = iou_xyxy(b0["xyxy"], b1["xyxy"])
                    if iou > best_iou:
                        best_iou, best_j = iou, j
                if best_j >= 0 and best_iou >= IOU_MATCH_THRESH:
                    c0, c1 = b0["conf"], fixed_boxes[best_j]["conf"]
                    if abs(c0 - c1) < DELTA_CONF:
                        winner = b0 if c0 >= c1 else fixed_boxes[best_j]
                        confused_rows.append({"cls":3,"conf":max(c0,c1),"xyxy":winner["xyxy"],"xywhn":winner["xywhn"]})
                        used0.add(i); used1.add(best_j)
                    else:
                        winner = b0 if c0 >= c1 else fixed_boxes[best_j]
                        if winner["conf"] >= MIN_CONF_PER_CLASS.get(winner["cls"],0.0):
                            keep_01.append(winner)
                        used0.add(i); used1.add(best_j)

            for i,b0 in enumerate(divot_boxes):
                if i not in used0 and b0["conf"]>=MIN_CONF_PER_CLASS[0]: keep_01.append(b0)
            for j,b1 in enumerate(fixed_boxes):
                if j not in used1 and b1["conf"]>=MIN_CONF_PER_CLASS[1]: keep_01.append(b1)

            all_boxes = keep_01 + confused_rows
            for b in other_boxes:
                thr = MIN_CONF_PER_CLASS.get(b["cls"],0.0)
                if b["conf"]>=thr: all_boxes.append(b)

            # Save labels
            label_path = ensure_label_path(img)
            rows = [(b["cls"], *b["xywhn"]) for b in all_boxes]
            wrote = write_yolo_txt(label_path, rows)

            # Visualization save
            if all_boxes:
                out_viz = VIZ_DIR / f"{img.stem}_viz.jpg"
                draw_boxes(img, all_boxes, out_viz)

            if wrote:
                images_labeled += 1
                total_labels += len(rows)
                for cls,*_ in rows:
                    per_class_counts[cls]+=1

        except Exception as e:
            print(f"[Skip] #{idx}: {img.name} ({e})")

        if idx % PROGRESS_EVERY==0 or idx==total:
            print(f"[Progress] {idx}/{total}")

    print("========== Auto-Labeling Summary ==========")
    print(f"Images scanned     : {total}")
    print(f"Images labeled     : {images_labeled}")
    print(f"Total labels saved : {total_labels}")
    for cid,name in CLASS_NAMES.items():
        print(f"Class {cid:>1} ({name:>16}): {per_class_counts[cid]}")
    print("================================================")

if __name__ == "__main__":
    main()


WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
[Start] Total images: 1179 (progress every 50)
Loading /home/dw/ws_job_msislab/Golf_Project/runs_yolo/20251124/20251107_data_yolov8s_img640_SGD_cls0.5_box7.5_dfl1.5_rectFalse_seed_33_20251122/weights/best_fp16_bs1_640px_static.engine for TensorRT inference...


[11/24/2025-12:35:49] [TRT] [W] WARNING The logger passed into createInferRuntime differs from one already registered for an existing builder, runtime, or refitter. So the current new logger is ignored, and TensorRT will use the existing one which is returned by nvinfer1::getLogger() instead.
[11/24/2025-12:35:49] [TRT] [I] Loaded engine size: 25 MiB
[11/24/2025-12:35:49] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +17, now: CPU 1, GPU 115 (MiB)
[Progress] 50/1179
[Progress] 100/1179
[Progress] 150/1179
[Progress] 200/1179
[Progress] 250/1179
[Progress] 300/1179
[Progress] 350/1179
[Progress] 400/1179
[Progress] 450/1179
[Progress] 500/1179
[Progress] 550/1179
[Progress] 600/1179
[Progress] 650/1179
[Progress] 700/1179
[Progress] 750/1179
[Progress] 800/1179
[Progress] 850/1179
[Progress] 900/1179
[Progress] 950/1179
[Progress] 1000/1179
[Progress] 1050/1179
[Progress] 1100/1179
[Progress] 1150/1179
[Progress] 1179/1179
========== A